# Merge Models with SLERP for `whats-up-doc`

## Install `mergekit`

In [1]:
!python3 -m pip install --upgrade pip
!git clone https://github.com/cg123/mergekit.git
!cd mergekit && pip install -q -e .

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
Cloning into 'mergekit'...
remote: Enumerating objects: 1928, done.
remote: Counting objects: 100% (1174/1174), done.
remote: Compressing objects: 100% (505/505), done.
remote: Total 1928 (delta 920), reused 766 (delta 668), pack-reused 754
Receiving objects: 100% (1928/1928), 572.57 KiB | 5.45 MiB/s, done.
Resolving deltas: 100% (1314/1314), done.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.9.3 requires keras-core, which is not installed.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.1.6 which is incompatible.
momepy 0

## Log-in to HuggingFace Using a Write Token

In [2]:
from huggingface_hub import notebook_login as hfl
hfl()

## Merged Model Metadata & Merge Config

In [15]:
MODEL_NAME = "Meta-Llama-3-8B-Instruct-Hermes-2-Pro-SLERP"
OUTPUT_DIR = "output-dir"

In [6]:
config = """
slices:
  - sources:
    - model: meta-llama/Meta-Llama-3-8B-Instruct
      layer_range: [0, 32]
    - model: NousResearch/Hermes-2-Pro-Llama-3-8B
      layer_range: [0, 32]
base_model: meta-llama/Meta-Llama-3-8B-Instruct
merge_method: slerp
parameters:
  t:
    - filter: self_attn
      value: [0, 0.5, 0.3, 0.7, 1]
    - filter: mlp
      value: [1, 0.5, 0.7, 0.3, 0]
    - value: 0.5
dtype: bfloat16
"""

In [7]:
with open("config.yaml", "w") as f:
    f.write(config)

## Perform Merge with `mergekit`

In [8]:
!mergekit-yaml config.yaml output-dir --allow-crimes --copy-tokenizer --out-shard-size 1B --low-cpu-memory --write-model-card --lazy-unpickle

Fetching 12 files:   0%|                                 | 0/12 [00:00<?, ?it/s]

dpo-adapter/adapter_config.json: 100%|█████████| 791/791 [00:00<00:00, 4.73MB/s]

Fetching 12 files:  17%|████▏                    | 2/12 [00:00<00:00, 13.19it/s]

model.safetensors.index.json: 100%|████████| 23.9k/23.9k [00:00<00:00, 83.9MB/s]


generation_config.json: 100%|███████████████████| 142/142 [00:00<00:00, 980kB/s]


model-00002-of-00004.safetensors:   0%|             | 0.00/5.00G [00:00<?, ?B/s]


model-00001-of-00004.safetensors:   0%|             | 0.00/4.98G [00:00<?, ?B/s]



model-00004-of-00004.safetensors:   0%|             | 0.00/1.17G [00:00<?, ?B/s]




model-00003-of-00004.safetensors:   0%|             | 0.00/4.92G [00:00<?, ?B/s]





dpo-adapter/adapter_model.safetensors:   0%|        | 0.00/2.27G [00:00<?, ?B/s]






special_tokens_map.json: 100%|█████████████████| 444/444 [00:00<00:00, 2.94MB/s]







tokenizer.json:   0%|                               | 0.00/9.09M [00:00<?, 

In [9]:
from kaggle_secrets import UserSecretsClient

userSecrets = UserSecretsClient()
HF_WRITE_TOKEN = userSecrets.get_secret("HF_WRITE_TOKEN")

In [ ]:
from huggingface_hub import HfApi

username = "0xVolt"

# Defined in the secrets tab in Kaggle Secrets
api = HfApi(token=HF_WRITE_TOKEN)

api.create_repo(
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)

# Push the whole merged model's folder to the hub
api.upload_folder(
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path=OUTPUT_DIR,
)

Upload 17 LFS files:   0%|          | 0/17 [00:00<?, ?it/s]

model-00002-of-00017.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

model-00001-of-00017.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

model-00004-of-00017.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

model-00003-of-00017.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

...

model-00013-of-00017.safetensors:   0%|          | 0.00/998M [00:00<?, ?B/s]

model-00014-of-00017.safetensors:   0%|          | 0.00/948M [00:00<?, ?B/s]

model-00015-of-00017.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

model-00016-of-00017.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

model-00017-of-00017.safetensors:   0%|          | 0.00/201M [00:00<?, ?B/s]